In [2]:
from openai import AssistantEventHandler
from typing_extensions import override
from openai import OpenAI

In [3]:
client = OpenAI(api_key="")

In [8]:

assistant = client.beta.assistants.create(
    name="Math Tutor 2",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o",
)


In [12]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print([f"assitant {a.id} and name: {a.name}" for a in my_assistants.data])

['assitant asst_6uGjzjpWWsDxji48eR4swmVf and name: Math Tutor 2', 'assitant asst_WMDfBOquGuD2ps4xXa5ZDU5Z and name: Math Tutor', 'assitant asst_6ZAy6JOCYLdI2WU3UNHfaU8c and name: Math Tutor', 'assitant asst_tH2T6XSwVFGNe7isUss4QSw2 and name: Math Tutor', 'assitant asst_ZffcrFABeWzbhFQ3ukBXO3bB and name: Math Tutor', 'assitant asst_ifPrzt2p7RLa2rjaqdgu1e3N and name: Math Tutor', 'assitant asst_nrEykEJvyeAnO2s8ZVCAYCVo and name: Math Tutor', 'assitant asst_tPgEDd5kX1c3tX9rgFfae2vT and name: None', 'assitant asst_cpHUsNuKNC1pIJHZkDh2RBTl and name: None', 'assitant asst_x77oShKZ6B3LPs5wWWGwKQ5b and name: None']


In [5]:

thread = client.beta.threads.create()

In [6]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)


# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

# Then, we use the `stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.


with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()




assistant > Of course, Jane Doe! To solve the equation \( 3x + 11 = 14 \), we need to isolate \( x \). Here are the steps:

1. Subtract 11 from both sides of the equation:
\[ 3x + 11 - 11 = 14 - 11 \]
\[ 3x = 3 \]

2. Divide both sides by 3 to solve for \( x \):
\[ \frac{3x}{3} = \frac{3}{3} \]
\[ x = 1 \]

So, the solution to the equation is \( x = 1 \).

Would you like this to be confirmed programmatically?